In [1]:
##########################################################################1########
##### Define all parameters for model tuning
##################################################################################

n_fold = 10
expName = "PSI_Site_DLNN_Custom_word2vec"
outPath = "Results"
foldName = "folds.pickle"

# modelNames = ["DLNN_3", "DLNN_5"]

epochs = 50
batch_size = 16
shuffle = False
seed = None

input_kmer_data_folder = "Data\\Psi_Site_Chen_Word2vec"

In [2]:
import os 
from Bio import SeqIO
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score

import math

In [3]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# ##################################################################################
# ##### define all CUSTOM functions
# ##################################################################################

# def one_hot_encode_dna(sequence):
    
#     seq_encoded = np.zeros((len(sequence),4))
#     dict_nuc = {
#         "A": 0,
#         "C": 1,
#         "G": 2,
#         "T":3
#     }
#     i = 0
    
#     for single_character in sequence:
#         if(single_character.upper() in dict_nuc.keys()):
#             seq_encoded[i][dict_nuc[single_character.upper()]] = 1
#             i = i+1
#         else:
#             return []
    
#     return seq_encoded

# def one_hot_encode_rna(sequence):
    
#     seq_encoded = np.zeros((len(sequence),4))
#     dict_nuc = {
#         "A": 0,
#         "C": 1,
#         "G": 2,
#         "U":3
#     }
#     i = 0
    
#     for single_character in sequence:
#         if(single_character.upper() in dict_nuc.keys()):
#             seq_encoded[i][dict_nuc[single_character.upper()]] = 1
#             i = i+1
#         else:
#             return []
    
#     return seq_encoded

In [5]:
##################################################################################
##### define evaluator functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

def pred2label(y_pred):
    y_pred = np.round(np.clip(y_pred, 0, 1))
    return y_pred

In [6]:
##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_custom_word2vec(input_shape = (64,),
                         dense_decode_units = 100, ## Dense layer parameters
                         prob = 0.25, learn_rate = 0.01, loss = 'binary_crossentropy', 
                         metrics = None):
    
    input1 = tf.keras.layers.Input(shape=input_shape)

    ##### LSTM Path
    #########################

#     x1 = tf.keras.layers.LSTM(lstm_decode_units, return_sequences = True)(input1)
#     x1 = tf.keras.layers.LSTM(1, return_sequences = True)(x1)
#     x1 = tf.keras.layers.Dropout(prob)(x1)
    
#     x1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(lstm_decode_units, return_sequences=True,
#                                                            kernel_regularizer = tf.keras.regularizers.l2(beta)))(input1)
    
#     model.add(Bidirectional(tf.keras.layers.LSTM(21)))
    
#     x1 = tf.keras.layers.Flatten()(x1)
#     x1 = tf.keras.layers.Dropout(prob)(x1)

    ##### Multipath Conv
    #########################
    
#     x2 = []
#     for kernel_length in range(2,max_kernel_length):
        

#         xx = tf.keras.layers.Conv1D(conv_filters_per_layer, kernel_length, strides = conv_strides,
#                                     kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
#         xx = tf.keras.layers.Activation('relu')(xx)
#         xx = tf.keras.layers.Flatten()(xx)
#         xx = tf.keras.layers.Dropout(prob)(xx)
        
#         x2.append(xx)
#     x2 = tf.keras.layers.MaxPooling1D(pool_size = max_pool_width, strides = max_pool_stride)(x2)
#     x2 = tf.keras.layers.Dropout(prob)(x2)

    ##### Fully connected Layers
    #########################
    
    y1 = tf.keras.layers.BatchNormalization()(input1)
    y1 = tf.keras.layers.Dense(dense_decode_units)(y1)
#     y1 = tf.keras.layers.Dropout(prob)(y1)
#     y1 = tf.keras.layers.BatchNormalization()(y1)
    y1 = tf.keras.layers.Dense(dense_decode_units*2)(y1)
#     y1 = tf.keras.layers.Dropout(prob)(y1)
    y1 = tf.keras.layers.Dense(dense_decode_units)(y1)
#     y1 = tf.keras.layers.Dropout(prob)(y1)
    y1 = tf.keras.layers.Dense(dense_decode_units/2)(y1)
#     x1 = tf.keras.layers.Dropout(prob)(y1)
    y1 = tf.keras.layers.Dense(1, activation = 'sigmoid')(y1)
#     y1 = tf.keras.layers.Dropout(prob)(y1)

#     y1 = tf.keras.layers.Dense(dense_decode_units)(y1)
    
#     y1 = tf.keras.layers.Dense(dense_decode_units/2)(y1)
#     y1 = tf.keras.layers.Dropout(prob)(y1)
    
#     y1 = tf.keras.layers.Dense(1, activation = 'sigmoid')(y1)
#     y1 = tf.keras.layers.Dense(1, activation = 'sigmoid')(y1)

    ##### Generate Model from input and output
    #########################
    
    model = tf.keras.models.Model(inputs=[input1], outputs=y1)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(lr=learn_rate), loss = loss)

    return model

In [7]:
DLNN_custom_word2vec().summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64)                256       
_________________________________________________________________
dense (Dense)                (None, 100)               6500      
_________________________________________________________________
dense_1 (Dense)              (None, 200)               20200     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51    

In [8]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Model" : [],
    "Kmer" : [],
    "Dataset" : [],
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

In [9]:
kmer_input_folders = list(set([x[0] for x in os.walk(input_kmer_data_folder)])-set([input_kmer_data_folder]))

for input_data_folder in kmer_input_folders:
    
    current_kmer = int(input_data_folder.split("\\")[-1][0])
    
    for root, dirs, files in os.walk(input_data_folder):
        for file in files:

            input_data_file = os.path.join(root, file)

            current_dataset_variety = file.split("\\")[-1].split(".")[0]

            ##################################################################################
            ##### extract data from the related embedding file
            ##################################################################################

            embedding = pd.read_csv(input_data_file)
            embedding = embedding.rename(columns={"Unnamed: 0" : "name"})
            embedding["class"] = pd.Series([1 if "P" in val else 0 for val in list(embedding["name"])])

            print("\n======================================================================")
            print("\nFile:", input_data_file)
            print("Positive:", embedding[embedding["class"] == 1].shape[0])
            print("Negative:", embedding[embedding["class"] == 0].shape[0])

            ##################################################################################
            ##### Generate Folds from dataset, and store to file
            ##################################################################################

            ## create the features and labels datasets for the training
            labels = np.array(embedding["class"])
            features = np.array(embedding.drop("name", axis=1).drop("class", axis=1))
            input_size = features[0].shape

            ## Generate the k-fold dataset
            folds = build_kfold(features, labels, k=n_fold, shuffle=shuffle, seed=seed)

            ## Write the k-fold dataset to file
            foldPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold))
            if(not os.path.isdir(foldPath)):
                os.makedirs(foldPath)
            pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

            ## Create and set directory to save model
            modelPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold), "models")
            if(not os.path.isdir(modelPath)):
                os.makedirs(modelPath)

            ##################################################################################
            ##### TRAIN and PREDICT for every Fold, using models
            ##################################################################################

            # fold counter
            i = 0

            for fold in folds:

                print("\nTrain/Test model "+current_dataset_variety+" on Fold #"+str(i)+".")

                kernel_length = 3
                ## Generate model using function
    #             model = Conv_LSTM_DLNN(input_shape = input_size, conv_filters_per_layer = 50, kernel_length = kernel_length, 
    #                                    lstm_decode_units = 50, max_pool_width = 2, max_pool_stride = 2, dense_decode_units = 50,
    #                                    learn_rate = 0.0001, prob = 0.5, loss='binary_crossentropy', metrics=None)

                model = DLNN_custom_word2vec(input_shape = input_size)

    #             ## Define the model callbacks for early stopping and saving the model. Then train model
    #             modelCallbacks = [
    #                 tf.keras.callbacks.ModelCheckpoint(os.path.join(modelPath, "{}_bestModel-fold{}.hdf5".format(current_dataset_variety, i)),
    #                                                    monitor = 'val_loss', verbose = 0, save_best_only = True, 
    #                                                    save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
    #                 tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 10, verbose = 0, 
    #                                                  mode = 'auto', baseline = None, restore_best_weights = False)
    #             ]
    #             model.fit(x = fold["X_train"], y = fold["y_train"], batch_size = batch_size, epochs = epochs, verbose = 0, 
    #                       callbacks = modelCallbacks, validation_split=0.2)

                model.fit(x = fold["X_train"], y = fold["y_train"], 
                          batch_size = batch_size, epochs = epochs, 
                          verbose = 1, 
                          validation_split=0.2)

                ##################################################################################
                ##### Prediction and metrics for TRAIN dataset
                ##################################################################################

                y_pred = model.predict(fold["X_train"])
                label_pred = pred2label(y_pred)
                # Compute precision, recall, sensitivity, specifity, mcc
                acc = accuracy_score(fold["y_train"], label_pred)
                prec = precision_score(fold["y_train"],label_pred)

                conf = confusion_matrix(fold["y_train"], label_pred)
                if(conf[0][0]+conf[1][0]):
                    sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
                else:
                    sens = 0.0
                if(conf[1][1]+conf[0][1]):
                    spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
                else:
                    spec = 0.0
                if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
                    mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
                else:
                    mcc= 0.0
                fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
                auc = roc_auc_score(fold["y_train"], y_pred)

                evaluations["Model"].append(current_dataset_variety)
                evaluations["Kmer"].append(current_kmer)
                evaluations["Dataset"].append(current_dataset_variety)
                evaluations["Fold"].append(i)
                evaluations["Train_Test"].append("Train")
                evaluations["Accuracy"].append(acc)
                evaluations["Precision"].append(prec)
                evaluations["TPR"].append(tpr)
                evaluations["FPR"].append(fpr)
                evaluations["TPR_FPR_Thresholds"].append(thresholds)
                evaluations["AUC"].append(auc)
                evaluations["Sensitivity"].append(sens)
                evaluations["Specificity"].append(spec)
                evaluations["MCC"].append(mcc)

                ##################################################################################
                ##### Prediction and metrics for TEST dataset
                ##################################################################################

                y_pred = model.predict(fold["X_test"])
                label_pred = pred2label(y_pred)
                # Compute precision, recall, sensitivity, specifity, mcc
                acc = accuracy_score(fold["y_test"], label_pred)
                prec = precision_score(fold["y_test"],label_pred)

                conf = confusion_matrix(fold["y_test"], label_pred)
                if(conf[0][0]+conf[1][0]):
                    sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
                else:
                    sens = 0.0
                if(conf[1][1]+conf[0][1]):
                    spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
                else:
                    spec = 0.0
                if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
                    mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
                else:
                    mcc= 0.0
                fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
                auc = roc_auc_score(fold["y_test"], y_pred)

                evaluations["Model"].append(current_dataset_variety)
                evaluations["Kmer"].append(current_kmer)
                evaluations["Dataset"].append(current_dataset_variety)
                evaluations["Fold"].append(i)
                evaluations["Train_Test"].append("Test")
                evaluations["Accuracy"].append(acc)
                evaluations["Precision"].append(prec)
                evaluations["TPR"].append(tpr)
                evaluations["FPR"].append(fpr)
                evaluations["TPR_FPR_Thresholds"].append(thresholds)
                evaluations["AUC"].append(auc)
                evaluations["Sensitivity"].append(sens)
                evaluations["Specificity"].append(spec)
                evaluations["MCC"].append(mcc)

                i = i+1
                del model
                tf.keras.backend.clear_session()

            ##################################################################################
            ##### Dump evaluations to a file
            ##################################################################################

            evalPath = os.path.join(outPath, expName, "_Evaluation_All_Datasets")
            if(not os.path.isdir(evalPath)):
                os.makedirs(evalPath)

            pickle.dump(evaluations,
                        open(os.path.join(evalPath, "{}fold_evaluations.pickle".format(n_fold)), "wb"))



File: Data\Psi_Site_Chen_Word2vec\4mer\HS_990.csv
Positive: 495
Negative: 495

Train/Test model HS_990 on Fold #0.
Epoch 1/50
45/45 [==============================] - 0s 7ms/step - loss: 0.6872 - val_loss: 0.7102
Epoch 2/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6728 - val_loss: 0.8303
Epoch 3/50
45/45 [==============================] - 0s 4ms/step - loss: 0.6741 - val_loss: 0.8429
Epoch 4/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6620 - val_loss: 1.3527
Epoch 5/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6661 - val_loss: 0.9196
Epoch 6/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6626 - val_loss: 0.9709
Epoch 7/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6634 - val_loss: 0.9915
Epoch 8/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6622 - val_loss: 0.9407
Epoch 9/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6631 - val_loss: 0.9993
E

45/45 [==============================] - 0s 4ms/step - loss: 0.6620 - val_loss: 0.9570
Epoch 32/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6630 - val_loss: 1.0016
Epoch 33/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6620 - val_loss: 0.9739
Epoch 34/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6618 - val_loss: 0.9880
Epoch 35/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6619 - val_loss: 0.9876
Epoch 36/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6623 - val_loss: 0.9774
Epoch 37/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6617 - val_loss: 0.9828
Epoch 38/50
45/45 [==============================] - 0s 4ms/step - loss: 0.6617 - val_loss: 0.9795
Epoch 39/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6618 - val_loss: 0.9743
Epoch 40/50
45/45 [==============================] - 0s 4ms/step - loss: 0.6619 - val_loss: 0.9727
Epoch 41/50
45/45 [===

45/45 [==============================] - 0s 4ms/step - loss: 0.6651 - val_loss: 0.9048
Epoch 14/50
45/45 [==============================] - 0s 4ms/step - loss: 0.6636 - val_loss: 0.9805
Epoch 15/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6638 - val_loss: 0.8943
Epoch 16/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6637 - val_loss: 0.9774
Epoch 17/50
45/45 [==============================] - 0s 4ms/step - loss: 0.6625 - val_loss: 0.9620
Epoch 18/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6622 - val_loss: 0.9553
Epoch 19/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6644 - val_loss: 0.9403
Epoch 20/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6628 - val_loss: 0.9923
Epoch 21/50
45/45 [==============================] - 0s 5ms/step - loss: 0.6623 - val_loss: 0.9683
Epoch 22/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6642 - val_loss: 0.9843
Epoch 23/50
45/45 [===

45/45 [==============================] - 0s 3ms/step - loss: 0.6645 - val_loss: 1.0417
Epoch 46/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6630 - val_loss: 1.0105
Epoch 47/50
45/45 [==============================] - 0s 4ms/step - loss: 0.6624 - val_loss: 0.9854
Epoch 48/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6626 - val_loss: 0.9481
Epoch 49/50
45/45 [==============================] - 0s 4ms/step - loss: 0.6624 - val_loss: 1.0277
Epoch 50/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6625 - val_loss: 1.0174

Train/Test model HS_990 on Fold #5.
Epoch 1/50
45/45 [==============================] - 0s 5ms/step - loss: 0.7030 - val_loss: 0.8245
Epoch 2/50
45/45 [==============================] - 0s 4ms/step - loss: 0.6737 - val_loss: 0.9026
Epoch 3/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6668 - val_loss: 0.8937
Epoch 4/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6650 - val_lo

45/45 [==============================] - 0s 4ms/step - loss: 0.6616 - val_loss: 0.9793
Epoch 28/50
45/45 [==============================] - 0s 4ms/step - loss: 0.6619 - val_loss: 0.9878
Epoch 29/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6619 - val_loss: 0.9857
Epoch 30/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6626 - val_loss: 0.9934
Epoch 31/50
45/45 [==============================] - 0s 4ms/step - loss: 0.6621 - val_loss: 0.9941
Epoch 32/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6619 - val_loss: 0.9774
Epoch 33/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6611 - val_loss: 0.9806
Epoch 34/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6613 - val_loss: 0.9739
Epoch 35/50
45/45 [==============================] - 0s 3ms/step - loss: 0.6619 - val_loss: 0.9744
Epoch 36/50
45/45 [==============================] - 0s 4ms/step - loss: 0.6613 - val_loss: 0.9617
Epoch 37/50
45/45 [===

...

29/29 [==============================] - 0s 4ms/step - loss: 0.6627 - val_loss: 0.9557
Epoch 33/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6633 - val_loss: 0.9743
Epoch 34/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6627 - val_loss: 0.9614
Epoch 35/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6632 - val_loss: 0.9622
Epoch 36/50
29/29 [==============================] - 0s 5ms/step - loss: 0.6665 - val_loss: 0.9984
Epoch 37/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6629 - val_loss: 0.9316
Epoch 38/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6633 - val_loss: 0.9732
Epoch 39/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6642 - val_loss: 0.9694
Epoch 40/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6628 - val_loss: 1.0030
Epoch 41/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6627 - val_loss: 0.9976
Epoch 42/50
29/29 [===

29/29 [==============================] - 0s 4ms/step - loss: 0.6615 - val_loss: 0.9539
Epoch 15/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6641 - val_loss: 0.9881
Epoch 16/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6627 - val_loss: 0.9581
Epoch 17/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6664 - val_loss: 0.9900
Epoch 18/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6628 - val_loss: 0.9746
Epoch 19/50
29/29 [==============================] - 0s 5ms/step - loss: 0.6653 - val_loss: 1.0043
Epoch 20/50
29/29 [==============================] - 0s 3ms/step - loss: 0.6628 - val_loss: 0.9604
Epoch 21/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6615 - val_loss: 0.9934
Epoch 22/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6624 - val_loss: 0.9827
Epoch 23/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6623 - val_loss: 0.9684
Epoch 24/50
29/29 [===

29/29 [==============================] - 0s 4ms/step - loss: 0.6621 - val_loss: 0.9968
Epoch 47/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6613 - val_loss: 0.9762
Epoch 48/50
29/29 [==============================] - 0s 5ms/step - loss: 0.6626 - val_loss: 0.9637
Epoch 49/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6632 - val_loss: 0.9768
Epoch 50/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6635 - val_loss: 0.9575

Train/Test model SS_628 on Fold #6.
Epoch 1/50
29/29 [==============================] - 0s 6ms/step - loss: 0.6961 - val_loss: 1.0140
Epoch 2/50
29/29 [==============================] - 0s 6ms/step - loss: 0.6813 - val_loss: 0.8561
Epoch 3/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6675 - val_loss: 1.0574
Epoch 4/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6613 - val_loss: 0.8534
Epoch 5/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6661 - val_los

29/29 [==============================] - 0s 4ms/step - loss: 0.6641 - val_loss: 0.9463
Epoch 29/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6636 - val_loss: 0.9527
Epoch 30/50
29/29 [==============================] - 0s 5ms/step - loss: 0.6621 - val_loss: 1.0071
Epoch 31/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6622 - val_loss: 0.9605
Epoch 32/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6621 - val_loss: 0.9867
Epoch 33/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6618 - val_loss: 0.9535
Epoch 34/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6618 - val_loss: 0.9803
Epoch 35/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6614 - val_loss: 0.9655
Epoch 36/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6614 - val_loss: 0.9836
Epoch 37/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6622 - val_loss: 1.0023
Epoch 38/50
29/29 [===

29/29 [==============================] - 0s 3ms/step - loss: 0.6796 - val_loss: 0.8945
Epoch 11/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6684 - val_loss: 0.9429
Epoch 12/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6647 - val_loss: 0.9207
Epoch 13/50
29/29 [==============================] - 0s 5ms/step - loss: 0.6612 - val_loss: 1.0242
Epoch 14/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6632 - val_loss: 0.9816
Epoch 15/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6629 - val_loss: 0.9855
Epoch 16/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6626 - val_loss: 0.9753
Epoch 17/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6638 - val_loss: 0.9849
Epoch 18/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6619 - val_loss: 0.9504
Epoch 19/50
29/29 [==============================] - 0s 4ms/step - loss: 0.6646 - val_loss: 0.9661
Epoch 20/50
29/29 [===

In [10]:
# for root, dirs, files in os.walk(input_data_folder):
#     for file in files:
        
#         input_data_file = os.path.join(root, file)
        
#         current_dataset_variety = file.split("\\")[-1].split(".")[0]
        
#         ##################################################################################
#         ##### extract data from the related embedding file
#         ##################################################################################
        
#         embedding = pd.read_csv(input_data_file)
#         embedding = embedding.rename(columns={"Unnamed: 0" : "name"})
#         embedding["class"] = pd.Series([1 if "P" in val else 0 for val in list(embedding["name"])])
        
#         print("\n======================================================================")
#         print("\nFile:", input_data_file)
#         print("Positive:", embedding[embedding["class"] == 1].shape[0])
#         print("Negative:", embedding[embedding["class"] == 0].shape[0])
        
#         ##################################################################################
#         ##### Generate Folds from dataset, and store to file
#         ##################################################################################

#         ## create the features and labels datasets for the training
#         labels = np.array(embedding["class"])
#         features = np.array(embedding.drop("name", axis=1).drop("class", axis=1))
#         input_size = features[0].shape

#         ## Generate the k-fold dataset
#         folds = build_kfold(features, labels, k=n_fold, shuffle=shuffle, seed=seed)

#         ## Write the k-fold dataset to file
#         foldPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold))
#         if(not os.path.isdir(foldPath)):
#             os.makedirs(foldPath)
#         pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

#         ## Create and set directory to save model
#         modelPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold), "models")
#         if(not os.path.isdir(modelPath)):
#             os.makedirs(modelPath)
            
#         ##################################################################################
#         ##### TRAIN and PREDICT for every Fold, using models
#         ##################################################################################

#         # fold counter
#         i = 0

#         for fold in folds:

#             print("\nTrain/Test model "+current_dataset_variety+" on Fold #"+str(i)+".")

#             kernel_length = 3
#             ## Generate model using function
# #             model = Conv_LSTM_DLNN(input_shape = input_size, conv_filters_per_layer = 50, kernel_length = kernel_length, 
# #                                    lstm_decode_units = 50, max_pool_width = 2, max_pool_stride = 2, dense_decode_units = 50,
# #                                    learn_rate = 0.0001, prob = 0.5, loss='binary_crossentropy', metrics=None)

#             model = DLNN_custom_word2vec(input_shape = input_size)

# #             ## Define the model callbacks for early stopping and saving the model. Then train model
# #             modelCallbacks = [
# #                 tf.keras.callbacks.ModelCheckpoint(os.path.join(modelPath, "{}_bestModel-fold{}.hdf5".format(current_dataset_variety, i)),
# #                                                    monitor = 'val_loss', verbose = 0, save_best_only = True, 
# #                                                    save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
# #                 tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 10, verbose = 0, 
# #                                                  mode = 'auto', baseline = None, restore_best_weights = False)
# #             ]
# #             model.fit(x = fold["X_train"], y = fold["y_train"], batch_size = batch_size, epochs = epochs, verbose = 0, 
# #                       callbacks = modelCallbacks, validation_split=0.2)

#             model.fit(x = fold["X_train"], y = fold["y_train"], 
#                       batch_size = batch_size, epochs = epochs, 
#                       verbose = 1, 
#                       validation_split=0.2)

#             ##################################################################################
#             ##### Prediction and metrics for TRAIN dataset
#             ##################################################################################

#             y_pred = model.predict(fold["X_train"])
#             label_pred = pred2label(y_pred)
#             # Compute precision, recall, sensitivity, specifity, mcc
#             acc = accuracy_score(fold["y_train"], label_pred)
#             prec = precision_score(fold["y_train"],label_pred)

#             conf = confusion_matrix(fold["y_train"], label_pred)
#             if(conf[0][0]+conf[1][0]):
#                 sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
#             else:
#                 sens = 0.0
#             if(conf[1][1]+conf[0][1]):
#                 spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
#             else:
#                 spec = 0.0
#             if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
#                 mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
#             else:
#                 mcc= 0.0
#             fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
#             auc = roc_auc_score(fold["y_train"], y_pred)

#             evaluations["Model"].append(current_dataset_variety)
#             evaluations["Kmer"].append(current_kmer)
#             evaluations["Dataset"].append(current_dataset_variety)
#             evaluations["Fold"].append(i)
#             evaluations["Train_Test"].append("Train")
#             evaluations["Accuracy"].append(acc)
#             evaluations["Precision"].append(prec)
#             evaluations["TPR"].append(tpr)
#             evaluations["FPR"].append(fpr)
#             evaluations["TPR_FPR_Thresholds"].append(thresholds)
#             evaluations["AUC"].append(auc)
#             evaluations["Sensitivity"].append(sens)
#             evaluations["Specificity"].append(spec)
#             evaluations["MCC"].append(mcc)

#             ##################################################################################
#             ##### Prediction and metrics for TEST dataset
#             ##################################################################################

#             y_pred = model.predict(fold["X_test"])
#             label_pred = pred2label(y_pred)
#             # Compute precision, recall, sensitivity, specifity, mcc
#             acc = accuracy_score(fold["y_test"], label_pred)
#             prec = precision_score(fold["y_test"],label_pred)

#             conf = confusion_matrix(fold["y_test"], label_pred)
#             if(conf[0][0]+conf[1][0]):
#                 sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
#             else:
#                 sens = 0.0
#             if(conf[1][1]+conf[0][1]):
#                 spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
#             else:
#                 spec = 0.0
#             if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
#                 mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
#             else:
#                 mcc= 0.0
#             fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
#             auc = roc_auc_score(fold["y_test"], y_pred)

#             evaluations["Model"].append(current_dataset_variety)
#             evaluations["Kmer"].append(current_kmer)
#             evaluations["Dataset"].append(current_dataset_variety)
#             evaluations["Fold"].append(i)
#             evaluations["Train_Test"].append("Test")
#             evaluations["Accuracy"].append(acc)
#             evaluations["Precision"].append(prec)
#             evaluations["TPR"].append(tpr)
#             evaluations["FPR"].append(fpr)
#             evaluations["TPR_FPR_Thresholds"].append(thresholds)
#             evaluations["AUC"].append(auc)
#             evaluations["Sensitivity"].append(sens)
#             evaluations["Specificity"].append(spec)
#             evaluations["MCC"].append(mcc)

#             i = i+1
#             del model
#             tf.keras.backend.clear_session()
            
#             raise ValueError("BLEH")

#         ##################################################################################
#         ##### Dump evaluations to a file
#         ##################################################################################

#         evalPath = os.path.join(outPath, expName, "_Evaluation_All_Datasets")
#         if(not os.path.isdir(evalPath)):
#             os.makedirs(evalPath)

#         pickle.dump(evaluations,
#                     open(os.path.join(evalPath, "{}fold_evaluations.pickle".format(n_fold)), "wb"))

## Visualization of Evaluation

In [11]:
##################################################################################
##### Add import statement here, to make this next part of code standalone executable
##################################################################################

import os
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
import numpy as np
import pandas as pd


In [12]:
##################################################################################
##### Load file and convert to dataframe for easy manipulation
##################################################################################

evalPath = os.path.join(outPath, expName, "_Evaluation_All_Datasets")
if(not os.path.isdir(evalPath)):
    os.makedirs(evalPath)

evaluations = pickle.load(open(os.path.join(evalPath, "{}fold_evaluations.pickle".format(n_fold)), "rb"))

In [13]:
# evaluations["Model"] = evaluations["Model"][0:20]
# evaluations_df = pd.DataFrame.from_dict(evaluations)

In [14]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

##################################################################################
##### Group dataset (mean of metrics) by [Dataset, Model, Train_Test] combinations
##################################################################################

evaluations_df_grouped = evaluations_df.groupby(["Dataset", 
                                                 "Model", 
                                                 "Kmer",
                                                 "Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

# DLNN_3 = evaluations_df_grouped[np.in1d(evaluations_df_grouped.index.get_level_values(1), ['DLNN_3'])]
# DLNN_5 = evaluations_df_grouped[np.in1d(evaluations_df_grouped.index.get_level_values(1), ['DLNN_5'])]

# DLNN_3_Train = DLNN_3[np.in1d(DLNN_3.index.get_level_values(2), ['Train'])]
# DLNN_3_Test = DLNN_3[np.in1d(DLNN_3.index.get_level_values(2), ['Test'])]

# DLNN_5_Train = DLNN_5[np.in1d(DLNN_5.index.get_level_values(2), ['Train'])]
# DLNN_5_Test = DLNN_5[np.in1d(DLNN_5.index.get_level_values(2), ['Test'])]

In [15]:
evaluations_df.columns

Index(['Model', 'Kmer', 'Dataset', 'Fold', 'Train_Test', 'Accuracy',
       'Precision', 'TPR', 'FPR', 'TPR_FPR_Thresholds', 'AUC', 'Sensitivity',
       'Specificity', 'MCC'],
      dtype='object')

In [16]:
evaluations_df_grouped

Accuracy  Precision       AUC  Sensitivity  \
Dataset Model  Kmer Train_Test                                               
HS_990  HS_990 3    Test        0.500000   0.500000  0.535592     0.000000   
                    Train       0.500000   0.500000  0.517077     0.000000   
               4    Test        0.500000   0.500000  0.489673     0.000000   
                    Train       0.500000   0.500000  0.514518     0.000000   
               5    Test        0.498990   0.449495  0.461224     0.049495   
                    Train       0.500112   0.450056  0.506572     0.050056   
MM_944  MM_944 3    Test        0.500000   0.500000  0.534519     0.000000   
                    Train       0.500000   0.500000  0.510989     0.000000   
               4    Test        0.500000   0.500000  0.524233     0.000000   
                    Train       0.500000   0.500000  0.501513     0.000000   
               5    Test        0.500000   0.500000  0.488801     0.000000   
                    Train       0.500000   0.500000  0.528303     0.000000   
SS_628  SS_628 3    Test        0.500000   0.500000  0.488820     0.000000   
                    Train       0.500000   0.500000  0.515478     0.000000   
               4    Test        0.500000   0.500000  0.523072     0.000000   
                    Train       0.500000   0.500000  0.517315     0.000000   
               5    Test        0.500000   0.500000  0.480196     0.000000   
                    Train       0.500000   0.500000  0.525493     0.000000   

                                Specificity  MCC  
Dataset Model  Kmer Train_Test                    
HS_990  HS_990 3    Test           0.500000  0.0  
                    Train          0.500000  0.0  
               4    Test           0.500000  0.0  
                    Train          0.500000  0.0  
               5    Test           0.449495  0.0  
                    Train          0.450056  0.0  
MM_944  MM_944 3    Test           0.500000  0.0  
                    Train          0.500000  0.0  
               4    Test           0.500000  0.0  
                    Train          0.500000  0.0  
               5    Test           0.500000  0.0  
                    Train          0.500000  0.0  
SS_628  SS_628 3    Test           0.500000  0.0  
                    Train          0.500000  0.0  
               4    Test           0.500000  0.0  
                    Train          0.500000  0.0  
               5    Test           0.500000  0.0  
                    Train          0.500000  0.0

In [17]:
# ##################################################################################
# ##### Decide on metric to visualize
# ##################################################################################

# print("Metrics Available : ")
# print(list(evaluations_df_grouped.columns))

#### Select a metric to plot below:

In [18]:
# metric_to_plot = "Accuracy"

In [19]:
# ##################################################################################
# ##### Visualize with a multiple Bar chart
# ##################################################################################

# x = np.arange(len(DLNN_3_Train[metric_to_plot]))
# width = 0.15

# fig, ax = plt.subplots(figsize=(17,6))
# rects1 = ax.bar(x - (4*(width/2)), round(DLNN_3_Train[metric_to_plot]*100, 3), width, label='DLNN_3, Train')
# rects2 = ax.bar(x - (1.5*(width/2)), round(DLNN_5_Train[metric_to_plot]*100, 3), width, label='DLNN_5, Train')
# rects3 = ax.bar(x + (1.5*(width/2)), round(DLNN_3_Test[metric_to_plot]*100, 3), width, label='DLNN_3, Test')
# rects4 = ax.bar(x + (4*(width/2)), round(DLNN_5_Test[metric_to_plot]*100, 3), width, label='DLNN_5, Test')

# ## Custom y-axis tick labels
# ax.set_ylabel(metric_to_plot)
# ax.set_ylim([(math.floor(min(evaluations_df_grouped[metric_to_plot])*10)-1)*10, 
#             (math.ceil(max(evaluations_df_grouped[metric_to_plot])*10)+1)*10])
# # ax.set_ylim([80, 105])

# ## Custom x-axis tick labels
# ax.set_xticks(x)
# # ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))
# # ax.set_xticklabels([m+" - "+str(n) for m,n in 
# #                         zip(DLNN_3_Train.index.get_level_values(0),DLNN_3_Train.index.get_level_values(1))],
# #                   rotation=30)
# ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))

# ax.set_title(metric_to_plot+' by Dataset, Model, Train/Test')
# ax.legend(loc='upper left')

# def autolabel(rects):
#     for rect in rects:
#         height = rect.get_height()
#         ax.annotate('{}'.format(height),
#                     xy=(rect.get_x() + rect.get_width() / 2, height),
#                     xytext=(0, 3),  # 3 points vertical offset
#                     textcoords="offset points", 
#                     ha='center', va='bottom', rotation=90)

# autolabel(rects1)
# autolabel(rects2)
# autolabel(rects3)
# autolabel(rects4)

# plt.show()

### Store all metrics' plots to file

In [20]:
# ##################################################################################
# ##### Iteratively generate comparison plot using every metric
# ##################################################################################

# for metric_to_plot in list(evaluations_df_grouped.columns):
    
#     x = np.arange(len(DLNN_3_Train[metric_to_plot]))
#     width = 0.15

#     fig, ax = plt.subplots(figsize=(17,6))
#     rects1 = ax.bar(x - (4*(width/2)), round(DLNN_3_Train[metric_to_plot]*100, 3), width, label='DLNN_3, Train')
#     rects2 = ax.bar(x - (1.5*(width/2)), round(DLNN_5_Train[metric_to_plot]*100, 3), width, label='DLNN_5, Train')
#     rects3 = ax.bar(x + (1.5*(width/2)), round(DLNN_3_Test[metric_to_plot]*100, 3), width, label='DLNN_3, Test')
#     rects4 = ax.bar(x + (4*(width/2)), round(DLNN_5_Test[metric_to_plot]*100, 3), width, label='DLNN_5, Test')

#     ## Custom y-axis tick labels
#     ax.set_ylabel(metric_to_plot)
#     ax.set_ylim([(math.floor(min(evaluations_df_grouped[metric_to_plot])*10)-1)*10, 
#                 (math.ceil(max(evaluations_df_grouped[metric_to_plot])*10)+1)*10])
#     # ax.set_ylim([80, 105])

#     ## Custom x-axis tick labels
#     ax.set_xticks(x)
#     # ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))
#     # ax.set_xticklabels([m+" - "+str(n) for m,n in 
#     #                         zip(DLNN_3_Train.index.get_level_values(0),DLNN_3_Train.index.get_level_values(1))],
#     #                   rotation=30)
#     ax.set_xticklabels(DLNN_3_Train.index.get_level_values(0))

#     ax.set_title(metric_to_plot+' by Dataset, Model, Train/Test')
#     ax.legend(loc='upper left')

#     def autolabel(rects):
#         for rect in rects:
#             height = rect.get_height()
#             ax.annotate('{}'.format(height),
#                         xy=(rect.get_x() + rect.get_width() / 2, height),
#                         xytext=(0, 3),  # 3 points vertical offset
#                         textcoords="offset points", 
#                         ha='center', va='bottom', rotation=90)

#     autolabel(rects1)
#     autolabel(rects2)
#     autolabel(rects3)
#     autolabel(rects4)
    
#     plt.savefig(os.path.join(evalPath, "{}_DLNN_Comparison".format(metric_to_plot)))
#     plt.close()
    